# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.4020427180880297

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (25 steps)
7.2e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.002 s/step ...done
MCMC, adapting ϵ (100 steps)
6.3e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
7.1e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
6.8e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
8.1e-6 s/step ...done
MCMC (1000 steps)
5.8e-6 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.00679348], -2.80667, 2, DoubledTurn, 0.990907, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.173617], -2.77899, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.158468], -2.74697, 2, DoubledTurn, 0.999102, 3), NUTS_Transition{Array{Float64,1},Float64}([0.0255109], -3.02296, 2, DoubledTurn, 0.938027, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.742001], -3.34038, 1, AdjacentTurn, 0.894586, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.348045], -3.04184, 1, AdjacentTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.583669], -2.9588, 1, DoubledTurn, 0.945354, 1), NUTS_Transition{Array{Float64,1},Float64}([-0.190969], -6.45981, 2, DoubledTurn, 0.502803, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.808583], -3.31517, 1, AdjacentTurn, 0.866549, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.808583], -3.67801, 1, DoubledTurn, 0.881609, 1)  …  NUTS_Trans

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

410.4101929643759

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.89, min/25%/median/75%/max: 0.12 0.84 0.96 1.0 1.0
  termination: AdjacentTurn => 28% DoubledTurn => 72%
  depth: 1 => 62% 2 => 38%


check the mean

In [11]:
mean(posterior_α)

0.452177630062822

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*